# Function Calling using Local LLMs (LlamaIndex)

### Table of Contents

1. **Load LLMs**
    * **Llama3 - 7B**
    * **Mistral - 7B**
2. **Define Functions / Tools**
3. **Create Agents**
    * **Llama3 Agent**
    * **Mistral Agent**

## 1. Load LLMs

In [1]:
from llama_index.llms.ollama import Ollama

llama3 = Ollama(model="llama3", base_url='http://localhost:11434', temperature=0.0, request_timeout=600)

llama3

Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f91bb7379a0>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7f91a9331480>, completion_to_prompt=<function default_completion_to_prompt at 0x7f91a93bdc60>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='llama3', temperature=0.0, context_window=3900, request_timeout=600.0, prompt_key='prompt', json_mode=False, additional_kwargs={})

In [2]:
from llama_index.llms.ollama import Ollama

mistral = Ollama(model="mistral", base_url='http://localhost:11434', temperature=0.0, request_timeout=600)

mistral

Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f90bde9b190>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7f91a9331480>, completion_to_prompt=<function default_completion_to_prompt at 0x7f91a93bdc60>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='mistral', temperature=0.0, context_window=3900, request_timeout=600.0, prompt_key='prompt', json_mode=False, additional_kwargs={})

In [3]:
mistral.metadata.is_chat_model, llama3.metadata.is_chat_model

(True, True)

## 2. Define Functions / Tools

In [4]:
from llama_index.core.tools import FunctionTool

def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

## 3. Define Agent

### 3.1 Llama3 Agent

**NOTE** - Function calling agent worker is specifically designed for function calling but currently, it does not work with **Ollama**. It'll work with openai, mistral, anthropic wrappers, only. Hence, I have used **ReAct** agent for this tutorial. In future, if it adds support for ollama then we can use it as well.

In [5]:
from llama_index.core.agent import ReActAgent, FunctionCallingAgentWorker

react_agent = ReActAgent.from_tools(tools=[multiply_tool, add_tool], llm=llama3, verbose=True)

react_agent

In [6]:
response = react_agent.chat("What is (121 + 2) * 5?")

print(str(response))

Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 121, 'b': 2}
Observation: 123
Thought: Now that we have the result of the addition (123), we can multiply it by 5 using another tool.
Action: multiply
Action Input: {'a': 123, 'b': 5}
Observation: 615
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result is 615.
The result is 615.


In [7]:
response.sources

[ToolOutput(content='123', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 2}}, raw_output=123, is_error=False),
 ToolOutput(content='615', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 123, 'b': 5}}, raw_output=615, is_error=False)]

In [8]:
for tool_output in response.sources:
    print(f"Tool : {tool_output.tool_name} - {tool_output.raw_input['kwargs']} - {tool_output.raw_output}")

Tool : add - {'a': 121, 'b': 2} - 123
Tool : multiply - {'a': 123, 'b': 5} - 615


In [9]:
react_agent.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is (121 + 2) * 5?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='The result is 615.', additional_kwargs={})]

In [10]:
for resoning_step in react_agent.get_completed_tasks()[0].extra_state["current_reasoning"]:
    print(f"{resoning_step.__class__.__name__} -> {resoning_step}\n")

ActionReasoningStep -> thought='The current language of the user is English. I need to use a tool to help me answer the question.' action='add' action_input={'a': 121, 'b': 2}

ObservationReasoningStep -> observation='123' return_direct=False

ActionReasoningStep -> thought='Now that we have the result of the addition (123), we can multiply it by 5 using another tool.' action='multiply' action_input={'a': 123, 'b': 5}

ObservationReasoningStep -> observation='615' return_direct=False

ResponseReasoningStep -> thought="I can answer without using any more tools. I'll use the user's language to answer" response='The result is 615.' is_streaming=False



In [11]:
print(f"System Prompt of Agent : \n\n {react_agent.get_prompts()['agent_worker:system_prompt'].template}")

System Prompt of Agent : 

 You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 

In [12]:
response = react_agent.query("Add 2 to 121 and then multiply result by 3")

print(str(response))

Thought: The current language of the user is not specified. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 121, 'b': 2}
Observation: 123
Thought: Now that we have the result of adding 2 to 121, which is 123, I need to multiply it by 3.
Action: multiply
Action Input: {'a': 123, 'b': 3}
Observation: 369
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Three hundred sixty-nine
Three hundred sixty-nine


In [13]:
response = react_agent.query("Add two to one hundred twenty one and then multiply result by seven")

print(str(response))

Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 121, 'b': 2}
Observation: 123
Thought: Now that we have the result of adding two to one hundred twenty one, which is 123, I need to multiply this by seven.
Action: multiply
Action Input: {'a': 123, 'b': 7}
Observation: 861
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Eight hundred sixty-one.
Eight hundred sixty-one.


In [14]:
(2+121)*7

861

In [15]:
response = react_agent.query("Add 1,675,213 to 3,124,562.")

print(str(response))

Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 1675213, 'b': 3124562}
Observation: 4799775
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result of adding 1,675,213 and 3,124,562 is 4,799,775.
The result of adding 1,675,213 and 3,124,562 is 4,799,775.


In [16]:
f"{1675213 + 3124562:,}"

'4,799,775'

### 3.2 Mistral Agent

In [17]:
from llama_index.core.agent import ReActAgent

react_agent2 = ReActAgent.from_tools(tools=[multiply_tool, add_tool], llm=mistral, verbose=True)

react_agent2

In [18]:
response = react_agent2.chat("What is (121 + 2) * 5?")

print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 121, 'b': 2}
Observation: 123
Thought: Now that I have the sum, I can use another tool to multiply the result by 5.
Action: multiply
Action Input: {'a': 123, 'b': 5}
Observation: 615
Thought: I can answer without using any more tools. I'll use English to answer.
Answer: The result is 615.
The result is 615.


In [19]:
for tool_output in response.sources:
    print(f"Tool : {tool_output.tool_name} - {tool_output.raw_input['kwargs']} - {tool_output.raw_output}")

Tool : add - {'a': 121, 'b': 2} - 123
Tool : multiply - {'a': 123, 'b': 5} - 615


In [20]:
response = react_agent2.query("Add 2 to 121 and then multiply result by 3")

print(str(response))

Thought: The current language of the user is: English. I need to use tools to help me answer the question.
Action: add
Action Input: {'a': 121, 'b': 2}
Observation: 123
Thought: Now that I have the sum, I need to multiply it by 3.
Action: multiply
Action Input: {'a': 123, 'b': 3}
Observation: 369
Thought: I can answer without using any more tools. I'll use English to answer.
Answer: The result of adding 2 to 121 and then multiplying the result by 3 is 369.
The result of adding 2 to 121 and then multiplying the result by 3 is 369.


In [21]:
response = react_agent2.query("Add two to one hundred twenty one and then multiply result by seven")

print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 1, 'b': 121}
Observation: 122
Thought: Now that I have the sum, I need to use another tool to multiply it by seven.
Action: multiply
Action Input: {'a': 122, 'b': 7}
Observation: 854
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result of adding two to one hundred twenty one and then multiplying it by seven is eight hundred fifty four.
The result of adding two to one hundred twenty one and then multiplying it by seven is eight hundred fifty four.


In [22]:
(2+121)*7

861

In [25]:
response = react_agent2.query("Add two to hundred twenty one and then multiply result by seven")

print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 2, 'b': 121}
Observation: 123
Thought: Now that I have the sum, I need to use another tool to multiply the result by seven.
Action: multiply
Action Input: {'a': 123, 'b': 7}
Observation: 861
Thought: I can answer without using any more tools. I'll use English to answer.
Answer: The sum of two and one hundred twenty-one, multiplied by seven is eight hundred sixty-one.
The sum of two and one hundred twenty-one, multiplied by seven is eight hundred sixty-one.


In [23]:
response = react_agent.query("Add 1,675,213 to 3,124,562.")

print(str(response))

Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 1675213, 'b': 3124562}
Observation: 4799775
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result of adding 1,675,213 and 3,124,562 is 4,799,775.
The result of adding 1,675,213 and 3,124,562 is 4,799,775.


In [24]:
f"{1675213 + 3124562:,}"

'4,799,775'

## Summary

In this video, I explained how to use **Local LLMs (Llama-3 7B & Mistral 7B)** through **Ollama** for function calling. We used **LlamaIndex** framework for our code.